In [1]:
%%init_spark
# Configure the location of the mesos master and spark distribution on HDFS
# launcher.jars = ["/some/local/path/to/a/file.jar"]
launcher.packages = ["org.elasticsearch:elasticsearch-hadoop:6.6.0",
                     "org.apache.hadoop:hadoop-aws:2.7.5",
                     "org.apache.hadoop:hadoop-common:2.7.5",
                     "com.amazonaws:aws-java-sdk:1.7.4"]
launcher.master = "k8s://kubernetes.default"
launcher.conf.set("spark.submit.deployMode", "client")
launcher.conf.set("spark.driver.port", "7077")
launcher.conf.set("spark.kubernetes.container.image", "ianchen06/spark-py:v2.4.3")
launcher.conf.set("spark.executor.instances", "2")
launcher.conf.set("spark.app.name", "ian")
launcher.conf.set("spark.kubernetes.namespace", "jupyterhub")
launcher.conf.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
launcher.conf.set("spark.hadoop.fs.s3a.endpoint", "http://10.30.0.151:9000")
launcher.conf.set("spark.hadoop.fs.s3a.access.key", "MZN9A1HBRJPS338J8VXI")
launcher.conf.set("spark.hadoop.fs.s3a.secret.key", "ePzAW5OAwIe8Z67ABWve9Jd4NYB+oHNGHabfAlzR")

In [2]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://jupyter-ian:4040
SparkContext available as 'sc' (version = 2.4.3, master = k8s://https://kubernetes.default, app id = spark-application-1558175637895)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@70fc9d4b


In [3]:
sc

res1: org.apache.spark.SparkContext = org.apache.spark.SparkContext@47eb98fb


In [4]:
val rdd = sc.parallelize(0 to 999)
rdd.takeSample(false, 5)

rdd: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[0] at parallelize at <console>:25
res2: Array[Int] = Array(589, 559, 787, 6, 430)


In [3]:
val reader = spark.read
  .format("org.elasticsearch.spark.sql")
  .option("es.nodes.wan.only","true")
  .option("es.port","9200")
  .option("es.net.ssl","false")
  .option("es.nodes", "10.30.0.121")

val df = reader.load("ptt-v1/doc")

reader: org.apache.spark.sql.DataFrameReader = org.apache.spark.sql.DataFrameReader@133cd85b
df: org.apache.spark.sql.DataFrame = [author: string, board: string ... 9 more fields]


In [11]:
df.count()

res6: Long = 183413


In [12]:
df.createTempView("ptt")

In [16]:
spark.sql("SELECT count(*) FROM ptt WHERE author IS NOT null LIMIT 10").show()

+--------+
|count(1)|
+--------+
|  182054|
+--------+



In [4]:
df.write.format("orc").save("s3a://spark/t")